# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
# Helium at the center of the box
atoms     = [ElementPsp(:He; psp=load_psp("hgh/lda/He-q2"))]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model = model_LDA(lattice, atoms, positions; symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
res   = self_consistent_field(basis; tol)
μref  = dipole(basis, res.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770503899547                   -0.53    9.0    160ms
  2   -2.771692842227       -2.92       -1.32    1.0   90.9ms
  3   -2.771714499217       -4.66       -2.50    1.0   93.1ms
  4   -2.771714659261       -6.80       -3.25    1.0   93.3ms
  5   -2.771714711866       -7.28       -3.77    2.0    110ms
  6   -2.771714715153       -8.48       -4.90    1.0   97.4ms
  7   -2.771714715249      -10.02       -5.34    2.0    113ms
  8   -2.771714715249      -12.24       -5.97    1.0    101ms
  9   -2.771714715250      -12.39       -7.05    2.0    132ms
 10   -2.771714715250   +  -13.89       -7.37    1.0    114ms
 11   -2.771714715250   +  -15.05       -8.14    1.0    104ms


-0.0001345740810362424

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_LDA(lattice, atoms, positions;
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε; tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770497661453                   -0.53    9.0    204ms
  2   -2.771776085227       -2.89       -1.31    1.0    103ms
  3   -2.771801883123       -4.59       -2.54    1.0    104ms
  4   -2.771802046660       -6.79       -3.41    1.0   95.4ms
  5   -2.771802073362       -7.57       -3.99    2.0    116ms
  6   -2.771802074450       -8.96       -5.03    1.0    101ms
  7   -2.771802074475      -10.60       -5.61    1.0    100ms
  8   -2.771802074476      -12.03       -6.07    2.0    124ms
  9   -2.771802074476      -13.48       -6.71    1.0    117ms
 10   -2.771802074476      -14.18       -7.51    2.0    128ms
 11   -2.771802074476      -14.27       -8.59    1.0    117ms


0.017612221478616612

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.0001345740810362424
Displaced dipole:  0.017612221478616612
Polarizability :   1.7746795559652855


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response
Now we use linear response to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field). Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

In [5]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; res.ρ)
    χ0δV = apply_χ0(res, δV)
    δρ - χ0δV
end

# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(res, δVext)

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve in iter 1; step 1: normres = 2.493920874874e-01
[ Info: GMRES linsolve in iter 1; step 2: normres = 3.766551727352e-03
[ Info: GMRES linsolve in iter 1; step 3: normres = 2.852769873648e-04
[ Info: GMRES linsolve in iter 1; step 4: normres = 4.694602730888e-06
[ Info: GMRES linsolve in iter 1; step 5: normres = 1.088786658687e-08
[ Info: GMRES linsolve in iter 1; step 6: normres = 6.277679428207e-11
[ Info: GMRES linsolve in iter 1; step 7: normres = 8.191884118523e-13
[ Info: GMRES linsolve in iter 1; finished at step 7: normres = 8.191884118523e-13
[ Info: GMRES linsolve in iter 2; step 1: normres = 6.652753854177e-11
[ Info: GMRES linsolve in iter 2; step 2: normres = 1.053373867600e-11
[ Info: GMRES linsolve in iter 2; step 3: normres = 2.273844523945e-13
[ Info: GMRES linsolve in iter 2; finished at step 3: normres = 2.273844523945e-13
┌ Info: GMRES linsolve converged at iteration 2, step 3:
│ *  norm of residual = 2.273992190185246e-13
└ *  number of operat

As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.